## Setup download data from the Data Registry

In [ ]:
# @title Data registry functions{ display-mode: "form" }
import requests

DATA_REGISTRY_BASE_URL = "https://data.open-contracting.org/en/"
PUBLICATIONS_URL = f"{DATA_REGISTRY_BASE_URL}publications.json"


def get_publications():
    publications = requests.get(PUBLICATIONS_URL, timeout=10).json()
    for publication in publications:
        publication["label"] = f"{publication['country']} - {publication['title']}"
    return publications


def get_publication_select_box():
    return widgets.Dropdown(
        options=sorted([entry["label"] for entry in get_publications()]),
        description="Publication:",
        disabled=False,
    )


def get_years_select_box(publication_select_box):
    selected_publication = next(
        filter(lambda entry: entry["label"] == publication_select_box.value, get_publications())
    )
    year_from = int(selected_publication["date_from"][:4])
    year_to = int(selected_publication["date_to"][:4])
    years = list(range(year_from, year_to + 1))
    years.append("full")
    return (
        widgets.Dropdown(
            options=years,
            description="Year:",
            disabled=False,
        ),
        selected_publication,
    )


def download_file(selected_publication, selected_year):
    file_name = ""
    download_url = f'{DATA_REGISTRY_BASE_URL}{selected_publication["id"]}/download?name={selected_year.value}.jsonl.gz'
    ! curl -sSOJ "$download_url"
    for file in os.listdir("."):
        if file.endswith(".gz"):
            file_name = file.replace(".gz", "")
            break
    ! gunzip -f "$file_name"
    ! echo "$file_name downloaded"
    return file_name